# WORK IN PROGRESS

currently just notes and goals

# Strava KOM Hunting

The goal of this notebook is to create a small, interactive dashboard that helps find "easy" top beat Strava KOMs. Motivation is twofold: 1) If you are not strong you better be smart, 2) Get to know Streamlit. Stretch goal: Estimate required Watts from segment info and sort by "Power to beat", then compare to my own power curve and show segments that I can beat.

Components:
- [stravalib](https://github.com/stravalib/stravalib) for API access to Strava
- `Parquet` for storing the API responses as the Strava API is limited in calls per day
- `DuckDB` as analytics engine
- `streamlit` to build the dashboard

Preamble: Most of the technology used is more complex than necessary for the size of data to process. The point is to explore these technologies so I can use them when they are necessary. Even building a database of all segments in a 100km radius around Munich I assume the file size to be in the low megabytes.

In [3]:
# Conda environment ds
import sys
print(sys.version)


3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0]


## 1. Connect to Strava API and get data.

- register Strava app to get API token
    - https://www.strava.com/settings/api with localhost as callback domain
- use refresh token after auth token expires (6h)
- get segment IDs by map are or radius around geographic location
- get segment info from IDs
- use bulk endpoints where possible to keep the API calls to a minimum

Tutorials and notes:
- https://medium.com/analytics-vidhya/accessing-user-data-via-the-strava-api-using-stravalib-d5bee7fdde17

Register Strava App and get the required secrets and tokens. This only needs to be done once. Secrets are stored in the non-tracked `config.py`.

In [2]:
from config import STRAVA_CLIENT_ID, STRAVA_CLIENT_SECRET
from stravalib.client import Client

client = Client()
authorize_url = client.authorization_url(client_id=STRAVA_CLIENT_ID, scope=['read_all', 'profile:read_all', 'activity:read_all'], redirect_uri="http://localhost:8282/authorized")

print(authorize_url) #extract response from code= parameter and add to config.py


https://www.strava.com/oauth/authorize?client_id=115409&redirect_uri=http%3A%2F%2Flocalhost%3A8282%2Fauthorized&approval_prompt=auto&scope=read_all%2Cprofile%3Aread_all%2Cactivity%3Aread_all&response_type=code


Exchange code for short-lived access token. Needs to be run everytime the token expires - currently 6h. Alternatively the `access_token` and `refresh_token` could be taken directly from the Strava API website https://www.strava.com/settings/api.

In [1]:
from config import STRAVA_CODE, STRAVA_CLIENT_ID, STRAVA_CLIENT_SECRET
from stravalib.client import Client

# register client
client = Client()

# get tokens and add to client
token_response = client.exchange_code_for_token(
    client_id=STRAVA_CLIENT_ID, client_secret=STRAVA_CLIENT_SECRET, code=STRAVA_CODE
)
client.access_token = token_response["access_token"]
client.refresh_token = token_response["refresh_token"]
client.token_expires_at = token_response["expires_at"]


If the token has expired get a new one using the refresh token.

In [ ]:
if time.time() > client.token_expires_at:
    refresh_response = client.refresh_access_token(
        client_id=STRAVA_CLIENT_ID, client_secret=STRAVA_CLIENT_SECRET, refresh_token=client.refresh_token
    )
    client.access_token = refresh_response["access_token"]
    client.refresh_token = refresh_response["refresh_token"]
    client.token_expires_at = refresh_response["expires_at"]
